# XORSTR Generic String Decryption
> Writing a generic string decryptor for this open source library

- toc: true 
- badges: true
- categories: [xorstr,decryption,python]

## Overview
The open source string encryption library [xorstr](https://github.com/JustasMasiulis/xorstr) has been adopted by multiple malware developers (as well as slight variations on the same technique). The string encryptor makes use of the `xmm`/`ymm` registers and `pxor` `pvxor` instructions to decrypt stack strings. In the words of the developer...

- All keys are 64bit and generated during compile time.
- Data blocks go in increments of 16 bytes so some space may be wasted.
- The code has been crafted so that all the data would be embedded directly into code and not stored on .rdata and such.
- The entirety of string encryption and decryption will be inlined.
 
 The following is an example of the library in use.

```
.text:00411E14 C7 44 24 08 25 7B 87 92                       mov     [esp+60h+var_58], 92877B25h
.text:00411E1C 0F 57 C0                                      xorps   xmm0, xmm0
.text:00411E1F C7 44 24 0C B6 10 A7 1F                       mov     [esp+60h+var_54], 1FA710B6h
.text:00411E27 8B 44 24 08                                   mov     eax, [esp+60h+var_58]
.text:00411E2B 8B 4C 24 0C                                   mov     ecx, [esp+60h+var_54]
.text:00411E2F 89 44 24 10                                   mov     dword ptr [esp+60h+var_50], eax
.text:00411E33 89 4C 24 14                                   mov     dword ptr [esp+60h+var_50+4], ecx
.text:00411E37 C7 44 24 08 D1 77 20 5B                       mov     [esp+60h+var_58], 5B2077D1h
.text:00411E3F C7 44 24 0C C5 36 32 7E                       mov     [esp+60h+var_54], 7E3236C5h
.text:00411E47 8B 44 24 08                                   mov     eax, [esp+60h+var_58]
.text:00411E4B 8B 4C 24 0C                                   mov     ecx, [esp+60h+var_54]
.text:00411E4F 89 44 24 18                                   mov     dword ptr [esp+60h+var_50+8], eax
.text:00411E53 89 4C 24 1C                                   mov     dword ptr [esp+60h+var_50+0Ch], ecx
.text:00411E57 C7 44 24 08 6D 1E EB FE                       mov     [esp+60h+var_58], 0FEEB1E6Dh
.text:00411E5F C7 44 24 0C D9 3C 87 48                       mov     [esp+60h+var_54], 48873CD9h
.text:00411E67 8B 44 24 08                                   mov     eax, [esp+60h+var_58]
.text:00411E6B 8B 4C 24 0C                                   mov     ecx, [esp+60h+var_54]
.text:00411E6F C7 44 24 08 BE 05 4C 3F                       mov     [esp+60h+var_58], 3F4C05BEh
.text:00411E77 89 44 24 40                                   mov     dword ptr [esp+60h+var_20], eax
.text:00411E7B C7 44 24 0C E4 36 32 7E                       mov     [esp+60h+var_54], 7E3236E4h
.text:00411E83 8B 44 24 08                                   mov     eax, [esp+60h+var_58]
.text:00411E87 89 4C 24 44                                   mov     dword ptr [esp+60h+var_20+4], ecx
.text:00411E8B 8B 4C 24 0C                                   mov     ecx, [esp+60h+var_54]
.text:00411E8F 89 44 24 48                                   mov     dword ptr [esp+60h+var_20+8], eax
.text:00411E93 8D 44 24 10                                   lea     eax, [esp+60h+var_50]
.text:00411E97 89 4C 24 4C                                   mov     dword ptr [esp+60h+var_20+0Ch], ecx
.text:00411E9B 8D 50 01                                      lea     edx, [eax+1]
.text:00411E9E 0F 28 4C 24 40                                movaps  xmm1, [esp+60h+var_20]
.text:00411EA3 66 0F EF 4C 24 10                             pxor    xmm1, [esp+60h+var_50]
.text:00411EA9 0F 29 4C 24 10                                movaps  [esp+60h+var_50], xmm1
.text:00411EAE 0F 29 44 24 20                                movaps  [esp+60h+var_40], xmm0
.text:00411EB3 C7 44 24 30 00 00 00 00                       mov     [esp+60h+var_30], 0
.text:00411EBB C7 44 24 34 00 00 00 00                       mov     [esp+60h+var_2C], 0
```

### Samples

- maybe cheat
  - `92394d5c170060b09ba4ffba450f44d5d4387693a00ee1aba910a818fa387b31`
- metastealer
  - `6cf8bfba1b221effcb1eccec0c91fb0906d0b8996932167f654680cb3ac53aac`
- meduza stealer x64
  - `2ad84bfff7d5257fdeb81b4b52b8e0115f26e8e0cdaa014f9e3084f518aa6149`
- meduza stealer x32
  - `29cf1ba279615a9f4c31d6441dd7c93f5b8a7d95f735c0daa3cc4dbb799f66d4`
- mpress unpacked risepro
  - `16ae203879efe1912bb8b97ceb0f4645abcde27a987e98a171d59f9c1ec3f764`
- privateloader
  - `1aa2d32ab883de5d4097a6d4fe7718a401f68ce95e0d2aea63212dd905103948`
- rise pro
  - `2cd2f077ca597ad0ef234a357ea71558d5e039da9df9958d0b8bd0efa92e74c9`
  

## Prior Work

There have been a few attempts at decrypting strings from malware that use this library (or a variation of the technique). While these are good references they don't cover all of the cases in a generic way which is our goal. There is also an IDA script that attempts to decrypt the library but we didn't have any success with it.

### X-Junior IDA Script
[X-Junior](https://github.com/X-Junior) has a script that we can try in IDA to decrypt these strings: [GitHub Repo](https://github.com/X-Junior/Malware-IDAPython-Scripts/tree/main/PivateLoader).

### Andre Tavares Python Script
[andretavare5](https://gist.github.com/andretavare5) has a python script using capstone to decrypt the strings: [Script Gist](https://gist.github.com/andretavare5/66ec413cdb4c7c39d35c22d38c7067a8#file-privateloader_str_decrypt-py). We have created our own hybrid of the two, which uses capstone for disassembly, but implements the logic from the IDA script.

### IDA Xorstr Decryption Plugin
[ida-jm-xorstr-decrypt-plugin](https://github.com/yubie-re/ida-jm-xorstr-decrypt-plugin). This plugin was made to directly attack the xorstr library but it is x64 only and in testing we could not get it to work reliably.

### aimware_deobf_str
[aimware_deobf_str](https://github.com/unknowntrojan/aimware_deobf_str) is a similar approach using RUST! lol.


## Regex and Dissassembly Approach

Our first attempt at decryption uses a regex to identify the `pxor`/`pvxor` instructions then disassembles a small chunk of surrounding code which is then scanned for the stack string `mov` immediate pattern. We also use a recursive search for `mov` where there is an intermediate register or temporary memory location used to store the immediate before it is moved into place.


### Decryption Algorithm
- select the first section in the PE file, assume this is the code
- scan code for final `pxor` instruction and truncate at this instruction to remove extra code from scanning (handle packers with large first sections)
- linear disassemble the prior 0x400 bytes of code - **not efficient**
- traverse assembly until `pxor` instruction is located
- scan backwards until all immediate data is located for the `xmm` registers
- decrypt xmm data, this is the string chunk

### Limitations
- In some cases we end up re-dissassembling the same code again and again
- The 0x400 bytes is an arbitrary number and might miss some of the string setup if there is a re-used chunk in a register (risepro)
- The dissassembly can sometimes be misaligned 
- The DWORD chunks are sometimes moved into place out of order, without tracking displacement we cannot correct this

### Possible Improvements
- Run full regex scan upfront and split binary into chunks based on the largest amount of code that covers all `pxor` instructions and only disassemble each once. 
- Use the `pxor` instruction offset to check for alignment when disassembling (still prone to errors... maybe try the x64dbg test for max valid instructions, could be slow)
- Instead of just scanning for `mov` instructions to collect the DWORD chunks use a recursive scan to track the displacement for each `mov`

**TODO**
- We still need to implement the optimization of only dissassembling big chunks of code the contain the strings instead of re-disassembling the same chunks again and again
- We need to implement EBP shift tracing 
- We need to check for other types of ESP shift and handle them
- We need to expand the xor types to handle more than pxor
- We need to expand the xor operands to handle more than just a register and an memory address 
- ****BUGS**** Currently we are missing a few strings still for rise.bin (check the sql statements)
- ****BUGS**** Our string builder sometimes adds strings that are seperate look for a better solution (tighter address spacing maybe, or something in the code???)
- ****BUGS**** Doesn't work for priv.bin likely just need to handle more xor operands

In [ ]:
def unhex(hex_string):
    import binascii
    if type(hex_string) == str:
        return binascii.unhexlify(hex_string.encode('utf-8'))
    else:
        return binascii.unhexlify(hex_string)

def tohex(data):
    import binascii
    if type(data) == str:
        return binascii.hexlify(data.encode('utf-8'))
    else:
        return binascii.hexlify(data)


In [74]:
import pefile
import struct
from capstone import *
from capstone.x86 import *
import re
import time
import logging

log_level = logging.ERROR

# Create logger
logger = logging.getLogger()
logger.setLevel(log_level)



# Create console handler and set level to error
ch = logging.StreamHandler()
ch.setLevel(log_level)

# Create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Add formatter to console handler
ch.setFormatter(formatter)

# Add console handler to logger
logger.addHandler(ch)

# Hack to clear handlers for jupyter notebook
for h in logger.handlers:
    logger.removeHandler(h)

def is_ascii(s):
    return all(c < 128 or c == 0 for c in s)


def xor(data, key):
    out = []
    for i in range(len(data)):
        out.append(data[i] ^ key[i % len(key)])
    return bytes(out)


def print_unique_strings(strings):
    string_dict = {}  
    last_string = ''
    for s in strings:
        if last_string != s[1]:
            string_dict[s[0]] = s[1]
        last_string = s[1]
    print(f"Found strings: {len(string_dict.keys())}\n")
    for o in string_dict.keys():
        print(f"{hex(o)} {string_dict[o]}")


def get_reg_data(instructions, reg_name):
    search_count = 0
    search_limit = 2000
    for inst in instructions:
        
        if search_count > search_limit:
            break
        search_count += 1
        
        if  inst.mnemonic == 'mov' and  inst.operands[0].type == X86_OP_REG  and  inst.operands[1].type == X86_OP_IMM:
            if inst.reg_name(inst.operands[0].reg) == reg_name:
                imm_value = inst.operands[1].value.imm
                data_chunk = struct.pack('<I',imm_value)
                return data_chunk
    return None


def get_data_from_pxor(instructions):
    data_chunks = []
    count = 0
    steps = 0
    steps_flag = 0
    flag_reg = 0
    search_count = 0
    search_limit = 400
    
    for inst_ptr in range(0,len(instructions)):
        inst = instructions[inst_ptr]
        steps +=1
        
        if search_count > search_limit:
            break
        search_count += 1
         
        if  inst.mnemonic == 'call':
            break

        if  inst.mnemonic == 'mov' and  inst.operands[0].type == X86_OP_REG  and  inst.operands[1].type == X86_OP_IMM:
            flag_reg = 1
        
        if  inst.mnemonic == 'mov' and ( (inst.operands[0].type == X86_OP_MEM and inst.operands[0].value.mem.disp != 0) or inst.operands[0].type == X86_OP_REG ) and inst.operands[1].type == X86_OP_IMM:
            imm_value = inst.operands[1].value.imm
            logger.debug(hex(imm_value))
            if imm_value & 0xff000000 == 0:
                break
            data_chunk = struct.pack('<I',imm_value)
            data_chunks.append(data_chunk)

            count += 1
            steps = 0
            steps_flag = 1

        if steps == 16 and steps_flag:
            break

        
    enc_data = data_chunks[0:count//2][::-1]
    key = data_chunks[count//2:count][::-1]
    
    if flag_reg :
        enc_data = sum(zip(enc_data[1::2], enc_data[::2]), ())
        key = sum(zip(key[1::2], key[::2]), ())
    return b''.join(enc_data),b''.join(key)


def get_imm_data_recursive(instructions, opr, displacement=0):
    logger.debug(f"get_imm_data_recursive: {opr}, displacement={displacement}")
    # Sanity check
    if opr.type != X86_OP_MEM and opr.type != X86_OP_REG:
        logger.debug(f"ERROR Operand type is not X86_OP_MEM or X86_OP_REG")
        return None

    # Determin the operand type
    # If the operand is a memory address search with displacement else search with register name
    for ptr in range(len(instructions)):
        inst = instructions[ptr]
        #logger.debug(f"recursive testing {inst} at {hex(inst.address)} ")

        # If the opr memory we are searching for is ESP then we need to check for stack changes
        if opr.type == X86_OP_MEM and opr.value.mem.base == X86_REG_ESP and inst.mnemonic == 'sub' and inst.operands[0].type == X86_OP_REG and inst.operands[0].reg == X86_REG_ESP:
            if inst.operands[1].type != X86_OP_IMM:
                logger.debug(f"ERROR: Expected immediate value for stack change")
                return None
            displacement -= inst.operands[1].value.imm
        
        if opr.type == X86_OP_MEM and opr.value.mem.base == X86_REG_ESP and inst.mnemonic == 'add' and inst.operands[0].type == X86_OP_REG and inst.operands[0].reg == X86_REG_ESP:
            if inst.operands[1].type != X86_OP_IMM:
                logger.debug(f"ERROR: Expected immediate value for stack change")
                return None
            displacement += inst.operands[1].value.imm
        
        # Track the movs to see if they have the memory/register we are looking for
        if inst.mnemonic == 'mov':
            if opr.type == X86_OP_MEM and inst.operands[0].type == X86_OP_MEM:
                inst_op_mem = inst.operands[0].value.mem
                if inst_op_mem.disp == opr.value.mem.disp + displacement and inst_op_mem.base == opr.value.mem.base and inst_op_mem.index == opr.value.mem.index and inst_op_mem.scale == opr.value.mem.scale:
                    if inst.operands[1].type == X86_OP_IMM:
                        return inst.operands[1].value.imm
                    else:
                        return get_imm_data_recursive(instructions[ptr:], inst.operands[1])
            elif opr.type == X86_OP_REG and inst.operands[0].type == X86_OP_REG:
                if inst.operands[0].reg == opr.reg:
                    if inst.operands[1].type == X86_OP_IMM:
                        return inst.operands[1].value.imm
                    else:
                        return get_imm_data_recursive(instructions[ptr:], inst.operands[1])
    return None
            

def get_string_from_pxor_ex(instructions, xor_reg, xor_mem):
    # Get the memory address moved into the xor_reg
    xor_reg_mem = None
    xor_reg_mem_offset = 0
    for ptr in range(len(instructions)):
        inst = instructions[ptr]
        logger.debug(f"Testing {hex(inst.address)}")
        if inst.mnemonic[:3] == 'mov' and inst.operands[0].type == X86_OP_REG and inst.operands[0].reg == xor_reg.reg:
            if inst.operands[1].type != X86_OP_MEM:
                logger.debug(f"Error mov xor_reg is not a memory address at {hex(inst.address)}: {inst.op_str}")
                return None
            else:
                # Get the memory address moved into the xor_mem
                logger.debug(f"Found mov xor_reg at {hex(inst.address)}: {inst.op_str}")
                xor_reg_mem = inst.operands[1]
                xor_reg_mem_offset = ptr
                break
    if xor_reg_mem is None:
        logger.debug(f"Error mov xor_reg not found")
        return None
    # Break xor_reg_mem memory base into 4 DWORD chunks
    logger.debug("Getting data chunks for xor_reg_mem")
    op_mem = xor_reg_mem.value.mem
    op_disp = op_mem.disp
    op_base = op_mem.base
    op_index = op_mem.index
    op_scale = op_mem.scale
    logger.debug(f"op_disp:{op_disp} op_base:{op_base} op_index:{op_index} op_scale:{op_scale}")

    data1 = b''
    for i in [0, 4, 8, 12]:
        tmp_chunk = get_imm_data_recursive(instructions[xor_reg_mem_offset:], xor_reg_mem, displacement=i)
        if tmp_chunk is None:
            logger.debug(f"Error no imm found for xor_reg_mem {xor_reg_mem} at displacement {i}")
            return None
        data1 += struct.pack('<I', tmp_chunk)   

    # Recursive scan for each chunk and get the imm value
    data2 = b''
    for i in [0, 4, 8, 12]:
        tmp_chunk = get_imm_data_recursive(instructions, xor_mem, displacement=i)
        if tmp_chunk is None:
            logger.debug(f"Error no imm found for xor_mem {xor_mem} at displacement {i}")
            return None
        data2 += struct.pack('<I', tmp_chunk)   
    
    out = xor(data1, data2)
    logger.debug(out)
    out = out.replace(b'\x00',b'')
    if len(out) == 0:
        return None
    #print(out.decode('utf-8'))

    if not is_ascii(out):
        return None
    
    return out.decode('utf-8')



def get_string_from_pxor(instructions):
    reversed_instruction_list = instructions[::-1]
    encrypted_str, key = get_data_from_pxor(reversed_instruction_list)

    if len(encrypted_str) == 0 or len(key) == 0:
        logger.debug(f"Error at {hex(inst.address)} key or data is missing")
        return None

    if len(encrypted_str) != len(key):
        logger.debug(f"Error at {hex(inst.address)} key and data not equal length")
        return None

    out = bytearray(encrypted_str[j] ^ key[j] for j in range(len(key)))

    #print(out)
    out = out.replace(b'\x00',b'')
    if len(out) == 0:
        return None
    #print(out.decode('utf-8'))

    if not is_ascii(out):
        return None
    
    return out.decode('utf-8')



def get_strings(filename):
    # Capstone setup
    md = Cs(CS_ARCH_X86, CS_MODE_32) 
    md.detail = True
    md.skipdata = True

    # Get code from PE
    pe = pefile.PE(filename)
    # Assume the first section is code
    txt = pe.sections[0]

    image_base = pe.OPTIONAL_HEADER.ImageBase
    section_rva = txt.VirtualAddress
    section_offset = txt.PointerToRawData
    section_data = txt.get_data()


    strings = []
        
    pxor_vpxor_vxorps_egg = rb'(\x66\x0F\xEF|\xC5\xFD\xEF|\xC5\xF8\x57)'

    for m in re.finditer(pxor_vpxor_vxorps_egg, section_data, re.DOTALL):
        xor_start = m.start() 

        # Determine the instruction length
        xor_instruction = list(md.disasm(section_data[xor_start:xor_start+0x10], image_base + section_rva + xor_start))[0]
        xor_instruction_address = image_base + section_rva + xor_start

        # if xor_instruction_address != 0x00463697:
        #     continue

        if xor_instruction.mnemonic != 'pxor' and xor_instruction.mnemonic != 'vpxor' and xor_instruction.mnemonic != 'vxorps':
            #print(f"Found {xor_instruction.mnemonic} instead of pxor")
            continue

        xor_len = xor_instruction.size
        if xor_instruction.operands[0].type == X86_OP_REG and xor_instruction.operands[1].type == X86_OP_REG:
            # Skip xor reg, reg
            continue
            
       
        if xor_instruction.operands[0].type != X86_OP_REG or xor_instruction.operands[1].type != X86_OP_MEM:
            # Skip anything that is not xor reg, [mem]
            continue
        
        # If we are here we have a xor reg, [mem] instruction
        #print(f"Found pxor at: {hex(xor_instruction.address)} len: {hex(xor_len)}")
        # op_mem = xor_instruction.operands[1].value.mem
        # op_disp = op_mem.disp
        # op_base = op_mem.base
        # op_index = op_mem.index
        # op_scale = op_mem.scale
        #print(f"Found pxor reg, [{op_base} + {op_index} * {op_scale} + {op_disp}]")
        #TODO update to use the new scanners
        
        scan_length = 0x2000
        instructions = []
        for inst in md.disasm(section_data[xor_start-scan_length:xor_start], xor_instruction_address - scan_length):
            instructions.append(inst)

        # # Old scanner
        # tmp_string = get_string_from_pxor(instructions)
        # if tmp_string is not None:
        #     strings.append((xor_instruction_address,tmp_string))

        if xor_instruction.mnemonic == 'pxor':
            logger.debug(f"Testing pxor at {hex(xor_instruction_address)}, {xor_instruction}")
            tmp_string = get_string_from_pxor_ex(instructions[::-1], xor_instruction.operands[0], xor_instruction.operands[1])
            if tmp_string is not None:
                strings.append((xor_instruction_address,tmp_string))

    return strings





filename = '/tmp/xorstr/work/rise.bin'

t = time.time()
strings = get_strings(filename)
# Benchmark 50.47584390640259 metastealer    
print(f"Benchmark {time.time() - t}")
print_unique_strings(strings)

       

print("done")


Benchmark 47.64587903022766
Found strings: 1103

0x4064ef RisePro
Telegra
0x40651e m: https://t.me/
0x40654e RiseProSUPPORT
0x458e88 winhttp.dll
0x458f88 wininet.dll
0x459123 LocalSimba
0x459463 grab_screen
0x4594d4 grab_tg
0x459542 grab_ds
0x4595b9 grab_wallets
0x459631 grab_ihistory
0x459717 logins
0x45977b Vault_IE
0x45982f logins
0x4598a4 WindowsCredentia
0x4598ff ls
0x459adb .zip
0x459df4 \screenshot.png
0x459e97 \Files
0x459f2f \FileZilla
0x45a0eb \Plugins
0x45a20b \
0x45a526 IndexedDB
0x45a59c Sync
0x45a606 Local
0x45a66e \
0x45a823 \Wallets
0x45a937 \
0x45ac41 IndexedDB
0x45acb7 Sync
0x45ad21 Local
0x45ad98 \
0x45afb0 \History
0x45b10f _
0x45b198 wb
0x45b1f1 .txt
0x45b25f \
0x45b464 url
0x45b56a time
0x45b7bc \CC
0x45b944 _
0x45b9d6 wb
0x45ba2f .txt
0x45ba9d \
0x45bddf nickname
0x45bed6 name_on_card
0x45bfd0 card_number
0x45c0f5 last_four
0x45c246 **** **** **** 
0x45c2e7 billing_address_
0x45c31a id
0x45c393 billing_address_
0x45c3e0 id
0x45c46c -
0x45c4d4 exp_month
0x45c5ce e

In [16]:

md = Cs(CS_ARCH_X86, CS_MODE_32) 
md.detail = True
md.skipdata = True
test = bytes.fromhex('898d34f8ffff898530f8ffffc78548f8ffff3ea0e7e0c7854cf8ffffd459e0a88b8548f8ffff8b8d4cf8ffff898d3cf8ffff898538f8ffffc78548f8ffff15d5541fc7854cf8ffff5910bb6b8b8548f8ffff8b8d4cf8ffffc78548f8ffff3ea0e7e0898540ffffff898d44ffffffc7854cf8ffffd459e0a88b8548f8ffff8b8d4cf8ffff0f288d30f8ffff')
for inst in md.disasm(test, 0):
    if inst.address == 0x6:   
        print(inst)
        print(inst.mnemonic)
        first = inst.operands[0]
    if inst.address == 0x84:   
        print(inst)
        print(inst.mnemonic)
        second = inst.operands[1]
        print(second)
        if first is second:
            print("same")
        else:
            print("not same")


<CsInsn 0x6 [898530f8ffff]: mov dword ptr [ebp - 0x7d0], eax>
mov
<CsInsn 0x84 [0f288d30f8ffff]: movaps xmm1, xmmword ptr [ebp - 0x7d0]>
movaps
not same


In [77]:
strings

last_addr = 0
for s in strings:
    print(f"{s[0] - last_addr}\t{s[1]}")
    last_addr = s[0] 


def string_builder(strings):
    out = []
    last_addr = 0
    last_string = ""
    for s in strings[::-1]:
        diff = last_addr - s[0]
        if diff <= 100 and last_string is not None:
            last_string = s[1] + last_string
        else:
            out.append((last_addr,last_string))
            last_string = s[1]
        last_addr = s[0]
    return out


ss= string_builder(strings)
print(f"Strings recovered: {len(ss)}")
print_unique_strings(ss)

4220143	RisePro
Telegra
47	m: https://t.me/
48	RiseProSUPPORT
338234	winhttp.dll
256	wininet.dll
411	LocalSimba
832	grab_screen
113	grab_tg
110	grab_ds
119	grab_wallets
120	grab_ihistory
230	logins
100	Vault_IE
180	logins
117	WindowsCredentia
91	ls
476	.zip
229	.zip
564	\screenshot.png
163	\Files
152	\FileZilla
177	\FileZilla
267	\Plugins
288	\
255	\
285	\
255	IndexedDB
118	Sync
106	Local
104	\
437	\Wallets
276	\
241	\
282	\
255	IndexedDB
118	Sync
106	Local
119	\
536	\History
351	_
137	wb
89	.txt
110	\
517	url
137	url
125	time
128	time
466	\CC
392	_
146	wb
89	.txt
110	\
834	nickname
126	nickname
121	name_on_card
129	name_on_card
121	card_number
129	card_number
164	last_four
129	last_four
208	**** **** **** 
161	billing_address_
51	id
121	billing_address_
77	id
140	-
104	exp_month
129	exp_month
121	exp_year
126	exp_year
110	expiration_month
172	expiration_month
196	expiration_year
118	expiration_year
145	-
1178	\Autofill
355	_
140	wb
77	.txt
89	\
519	name
128	name
125	value
131	value
45

In [60]:
print(X86_REG_ESP)

30


16